In [4]:
import numpy as np
import pandas as pd
from pandas import json_normalize
import ast
from datetime import timedelta
import holidays


pd.set_option('display.max_colwidth', None)

Read the datasets into a pandas df

In [5]:
charging_sessions_orig = pd.read_csv("./data/original/charging_sessions.csv")
weather_orig = pd.read_csv("./data/original/weather_burbank_airport.csv")

Start with the charging sessions, lets get some general information about the dataset.

In [6]:
charging_sessions_orig.info()
charging_sessions_orig.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66450 entries, 0 to 66449
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        66450 non-null  int64  
 1   id                66450 non-null  object 
 2   connectionTime    66450 non-null  object 
 3   disconnectTime    66450 non-null  object 
 4   doneChargingTime  62362 non-null  object 
 5   kWhDelivered      66450 non-null  float64
 6   sessionID         66450 non-null  object 
 7   siteID            66450 non-null  int64  
 8   spaceID           66450 non-null  object 
 9   stationID         66450 non-null  object 
 10  timezone          66450 non-null  object 
 11  userID            49187 non-null  float64
 12  userInputs        49187 non-null  object 
dtypes: float64(2), int64(2), object(9)
memory usage: 6.6+ MB


,Unnamed: 0,kWhDelivered,siteID,userID
count,66450.000000,66450.000000,66450.000000,49187.000000
mean,4726.102062,11.787916,1.472656,1800.494582
std,3568.025394,10.314789,0.499256,2617.548614
min,0.000000,0.501000,1.000000,1.000000
25%,1845.000000,5.097000,1.000000,431.000000
50%,3960.000000,9.140000,1.000000,818.000000
75%,7027.000000,14.183000,2.000000,1805.000000
max,15291.000000,108.797242,2.000000,19923.000000


In [7]:
charging_sessions_orig.head(5)

,Unnamed: 0,id,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,sessionID,siteID,spaceID,stationID,timezone,userID,userInputs
0,0,5e23b149f9af8b5fe4b973cf,2020-01-02 13:08:54+00:00,2020-01-02 19:11:15+00:00,2020-01-02 17:31:35+00:00,25.016,1_1_179_810_2020-01-02 13:08:53.870034,1,AG-3F30,1-1-179-810,America/Los_Angeles,194.0,"[{'WhPerMile': 250, 'kWhRequested': 25.0, 'milesRequested': 100, 'minutesAvailable': 463, 'modifiedAt': 'Thu, 02 Jan 2020 13:09:39 GMT', 'paymentRequired': True, 'requestedDeparture': 'Thu, 02 Jan 2020 20:51:54 GMT', 'userID': 194}]"
1,1,5e23b149f9af8b5fe4b973d0,2020-01-02 13:36:50+00:00,2020-01-02 22:38:21+00:00,2020-01-02 20:18:05+00:00,33.097,1_1_193_825_2020-01-02 13:36:49.599853,1,AG-1F01,1-1-193-825,America/Los_Angeles,4275.0,"[{'WhPerMile': 280, 'kWhRequested': 70.0, 'milesRequested': 250, 'minutesAvailable': 595, 'modifiedAt': 'Thu, 02 Jan 2020 13:37:11 GMT', 'paymentRequired': True, 'requestedDeparture': 'Thu, 02 Jan 2020 23:31:50 GMT', 'userID': 4275}]"
2,2,5e23b149f9af8b5fe4b973d1,2020-01-02 13:56:35+00:00,2020-01-03 00:39:22+00:00,2020-01-02 16:35:06+00:00,6.521,1_1_193_829_2020-01-02 13:56:35.214993,1,AG-1F03,1-1-193-829,America/Los_Angeles,344.0,"[{'WhPerMile': 400, 'kWhRequested': 8.0, 'milesRequested': 20, 'minutesAvailable': 60, 'modifiedAt': 'Thu, 02 Jan 2020 13:57:17 GMT', 'paymentRequired': True, 'requestedDeparture': 'Thu, 02 Jan 2020 14:56:35 GMT', 'userID': 344}]"
3,3,5e23b149f9af8b5fe4b973d2,2020-01-02 13:59:58+00:00,2020-01-02 16:38:39+00:00,2020-01-02 15:18:45+00:00,2.355,1_1_193_820_2020-01-02 13:59:58.309319,1,AG-1F04,1-1-193-820,America/Los_Angeles,1117.0,"[{'WhPerMile': 400, 'kWhRequested': 8.0, 'milesRequested': 20, 'minutesAvailable': 65, 'modifiedAt': 'Thu, 02 Jan 2020 14:00:03 GMT', 'paymentRequired': True, 'requestedDeparture': 'Thu, 02 Jan 2020 15:04:58 GMT', 'userID': 1117}, {'WhPerMile': 400, 'kWhRequested': 8.0, 'milesRequested': 20, 'minutesAvailable': 65, 'modifiedAt': 'Thu, 02 Jan 2020 14:00:19 GMT', 'paymentRequired': True, 'requestedDeparture': 'Thu, 02 Jan 2020 15:04:58 GMT', 'userID': 1117}]"
4,4,5e23b149f9af8b5fe4b973d3,2020-01-02 14:00:01+00:00,2020-01-02 22:08:40+00:00,2020-01-02 18:17:30+00:00,13.375,1_1_193_819_2020-01-02 14:00:00.779967,1,AG-1F06,1-1-193-819,America/Los_Angeles,334.0,"[{'WhPerMile': 400, 'kWhRequested': 16.0, 'milesRequested': 40, 'minutesAvailable': 504, 'modifiedAt': 'Thu, 02 Jan 2020 14:00:13 GMT', 'paymentRequired': True, 'requestedDeparture': 'Thu, 02 Jan 2020 22:24:01 GMT', 'userID': 334}]"


In [8]:
charging_sessions_orig.nunique()

Unnamed: 0          15292
id                  65037
connectionTime      64839
disconnectTime      64906
doneChargingTime    60637
kWhDelivered        25629
sessionID           65037
siteID                  2
spaceID               107
stationID             107
timezone                1
userID               1006
userInputs          47822
dtype: int64

Judging by the number of unique id's in comparison to the whole dataset, we have duplicates. Let's drop them

In [9]:
charging_sessions_modified = charging_sessions_orig.drop_duplicates()

In [10]:
charging_sessions_modified.info()
charging_sessions_modified.nunique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66450 entries, 0 to 66449
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        66450 non-null  int64  
 1   id                66450 non-null  object 
 2   connectionTime    66450 non-null  object 
 3   disconnectTime    66450 non-null  object 
 4   doneChargingTime  62362 non-null  object 
 5   kWhDelivered      66450 non-null  float64
 6   sessionID         66450 non-null  object 
 7   siteID            66450 non-null  int64  
 8   spaceID           66450 non-null  object 
 9   stationID         66450 non-null  object 
 10  timezone          66450 non-null  object 
 11  userID            49187 non-null  float64
 12  userInputs        49187 non-null  object 
dtypes: float64(2), int64(2), object(9)
memory usage: 6.6+ MB


Unnamed: 0          15292
id                  65037
connectionTime      64839
disconnectTime      64906
doneChargingTime    60637
kWhDelivered        25629
sessionID           65037
siteID                  2
spaceID               107
stationID             107
timezone                1
userID               1006
userInputs          47822
dtype: int64

This didnt quite do the job, we still have the same amount of entries in the dataset. We need to find out where they differ. But first, let's drop the timezone, since its the same for all of the data.
We also have no information about the the Unnamed: 0 column. It looks to be some kind of identifier, but doesn't have enough unique values to uniquely identify all data points. Since we have no further information, assume this is a data fragment of whoever created the dataset, that we weren't supposed to get. We will drop it too.  

In [11]:
charging_sessions_modified = charging_sessions_orig.drop(columns = ["Unnamed: 0","timezone"])

In [12]:
charging_sessions_modified.info()
charging_sessions_modified.nunique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66450 entries, 0 to 66449
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                66450 non-null  object 
 1   connectionTime    66450 non-null  object 
 2   disconnectTime    66450 non-null  object 
 3   doneChargingTime  62362 non-null  object 
 4   kWhDelivered      66450 non-null  float64
 5   sessionID         66450 non-null  object 
 6   siteID            66450 non-null  int64  
 7   spaceID           66450 non-null  object 
 8   stationID         66450 non-null  object 
 9   userID            49187 non-null  float64
 10  userInputs        49187 non-null  object 
dtypes: float64(2), int64(1), object(8)
memory usage: 5.6+ MB


id                  65037
connectionTime      64839
disconnectTime      64906
doneChargingTime    60637
kWhDelivered        25629
sessionID           65037
siteID                  2
spaceID               107
stationID             107
userID               1006
userInputs          47822
dtype: int64

In [13]:
# Filter out all rows where id is a duplicate
duplicates = charging_sessions_modified[charging_sessions_modified.duplicated("id", keep = False)]

In [14]:
duplicates.sort_values(by = "sessionID")

,id,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,sessionID,siteID,spaceID,stationID,userID,userInputs
65046,5d310f54f9af8b52adda3e28,2019-07-02 13:25:01+00:00,2019-07-02 22:39:18+00:00,2019-07-02 16:29:23+00:00,13.924,1_1_178_817_2019-07-02 13:25:01.464654,1,AG-1F09,1-1-178-817,322.0,"[{'WhPerMile': 313, 'kWhRequested': 18.78, 'milesRequested': 60, 'minutesAvailable': 418, 'modifiedAt': 'Tue, 02 Jul 2019 13:25:07 GMT', 'paymentRequired': True, 'requestedDeparture': 'Tue, 02 Jul 2019 20:23:01 GMT', 'userID': 322}]"
5582,5d310f54f9af8b52adda3e28,2019-07-02 13:25:01+00:00,2019-07-02 22:39:18+00:00,2019-07-02 16:29:23+00:00,13.924,1_1_178_817_2019-07-02 13:25:01.464654,1,AG-1F09,1-1-178-817,322.0,"[{'WhPerMile': 313, 'kWhRequested': 18.78, 'milesRequested': 60, 'minutesAvailable': 418, 'modifiedAt': 'Tue, 02 Jul 2019 13:25:07 GMT', 'paymentRequired': True, 'requestedDeparture': 'Tue, 02 Jul 2019 20:23:01 GMT', 'userID': 322}]"
65124,5d3260aff9af8b56e4697230,2019-07-03 16:20:04+00:00,2019-07-03 22:41:16+00:00,2019-07-03 20:37:22+00:00,11.184,1_1_178_817_2019-07-03 16:20:03.809665,1,AG-1F09,1-1-178-817,357.0,"[{'WhPerMile': 354, 'kWhRequested': 14.16, 'milesRequested': 40, 'minutesAvailable': 238, 'modifiedAt': 'Wed, 03 Jul 2019 16:21:04 GMT', 'paymentRequired': True, 'requestedDeparture': 'Wed, 03 Jul 2019 20:18:04 GMT', 'userID': 357}]"
5660,5d3260aff9af8b56e4697230,2019-07-03 16:20:04+00:00,2019-07-03 22:41:16+00:00,2019-07-03 20:37:22+00:00,11.184,1_1_178_817_2019-07-03 16:20:03.809665,1,AG-1F09,1-1-178-817,357.0,"[{'WhPerMile': 354, 'kWhRequested': 14.16, 'milesRequested': 40, 'minutesAvailable': 238, 'modifiedAt': 'Wed, 03 Jul 2019 16:21:04 GMT', 'paymentRequired': True, 'requestedDeparture': 'Wed, 03 Jul 2019 20:18:04 GMT', 'userID': 357}]"
5694,5d38f852f9af8b69fe3549fd,2019-07-08 13:09:26+00:00,2019-07-08 19:03:10+00:00,2019-07-08 17:49:27+00:00,11.093,1_1_178_817_2019-07-08 13:08:13.199752,1,AG-1F09,1-1-178-817,2500.0,"[{'WhPerMile': 250, 'kWhRequested': 17.5, 'milesRequested': 70, 'minutesAvailable': 475, 'modifiedAt': 'Mon, 08 Jul 2019 13:09:25 GMT', 'paymentRequired': True, 'requestedDeparture': 'Mon, 08 Jul 2019 21:04:26 GMT', 'userID': 2500}]"
...,...,...,...,...,...,...,...,...,...,...,...
66320,5d55f950f9af8b45dfb3d875,2019-07-30 11:31:17+00:00,2019-07-30 22:07:15+00:00,2019-07-30 20:57:54+00:00,46.863,1_1_194_826_2019-07-30 11:31:16.894598,1,AG-1F11,1-1-194-826,933.0,"[{'WhPerMile': 345, 'kWhRequested': 62.1, 'milesRequested': 180, 'minutesAvailable': 624, 'modifiedAt': 'Tue, 30 Jul 2019 11:31:40 GMT', 'paymentRequired': True, 'requestedDeparture': 'Tue, 30 Jul 2019 21:55:17 GMT', 'userID': 933}]"
66382,5d55f950f9af8b45dfb3d8b3,2019-07-30 22:30:46+00:00,2019-07-31 00:44:02+00:00,2019-07-31 00:45:01+00:00,4.166,1_1_194_826_2019-07-30 22:30:45.743815,1,AG-1F11,1-1-194-826,76.0,"[{'WhPerMile': 301, 'kWhRequested': 24.08, 'milesRequested': 80, 'minutesAvailable': 122, 'modifiedAt': 'Tue, 30 Jul 2019 22:31:09 GMT', 'paymentRequired': True, 'requestedDeparture': 'Wed, 31 Jul 2019 00:32:46 GMT', 'userID': 76}]"
6918,5d55f950f9af8b45dfb3d8b3,2019-07-30 22:30:46+00:00,2019-07-31 00:44:02+00:00,2019-07-31 00:45:01+00:00,4.166,1_1_194_826_2019-07-30 22:30:45.743815,1,AG-1F11,1-1-194-826,76.0,"[{'WhPerMile': 301, 'kWhRequested': 24.08, 'milesRequested': 80, 'minutesAvailable': 122, 'modifiedAt': 'Tue, 30 Jul 2019 22:31:09 GMT', 'paymentRequired': True, 'requestedDeparture': 'Wed, 31 Jul 2019 00:32:46 GMT', 'userID': 76}]"
66393,5d574ad2f9af8b4c10c0361e,2019-07-31 11:57:10+00:00,2019-07-31 21:20:17+00:00,2019-07-31 19:08:13+00:00,25.971,1_1_194_826_2019-07-31 11:57:09.786222,1,AG-1F11,1-1-194-826,933.0,"[{'WhPerMile': 345, 'kWhRequested': 41.4, 'milesRequested': 120, 'minutesAvailable': 624, 'modifiedAt': 'Wed, 31 Jul 2019 11:57:25 GMT', 'paymentRequired': True, 'requestedDeparture': 'Wed, 31 Jul 2019 22:21:10 GMT', 'userID': 933}]"


The duplicate rows look exactly the same, let's try dropping duplicates again.

In [15]:
charging_sessions_modified.drop_duplicates(inplace = True)

In [16]:
charging_sessions_modified.info()
charging_sessions_modified.nunique()

<class 'pandas.core.frame.DataFrame'>
Index: 65037 entries, 0 to 65036
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                65037 non-null  object 
 1   connectionTime    65037 non-null  object 
 2   disconnectTime    65037 non-null  object 
 3   doneChargingTime  60950 non-null  object 
 4   kWhDelivered      65037 non-null  float64
 5   sessionID         65037 non-null  object 
 6   siteID            65037 non-null  int64  
 7   spaceID           65037 non-null  object 
 8   stationID         65037 non-null  object 
 9   userID            47822 non-null  float64
 10  userInputs        47822 non-null  object 
dtypes: float64(2), int64(1), object(8)
memory usage: 6.0+ MB


id                  65037
connectionTime      64839
disconnectTime      64906
doneChargingTime    60637
kWhDelivered        25629
sessionID           65037
siteID                  2
spaceID               107
stationID             107
userID               1006
userInputs          47822
dtype: int64

That did the job, we now have the same amount of unique id's as we have total rows.
We can now safely drop the id column. Since the sessionID is an aggregate of the connection time and station id, we will drop it too. 

In [17]:
charging_sessions_modified.drop(columns = ["id", "sessionID"], inplace = True)

In [18]:
charging_sessions_modified.head()

,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,siteID,spaceID,stationID,userID,userInputs
0,2020-01-02 13:08:54+00:00,2020-01-02 19:11:15+00:00,2020-01-02 17:31:35+00:00,25.016,1,AG-3F30,1-1-179-810,194.0,"[{'WhPerMile': 250, 'kWhRequested': 25.0, 'milesRequested': 100, 'minutesAvailable': 463, 'modifiedAt': 'Thu, 02 Jan 2020 13:09:39 GMT', 'paymentRequired': True, 'requestedDeparture': 'Thu, 02 Jan 2020 20:51:54 GMT', 'userID': 194}]"
1,2020-01-02 13:36:50+00:00,2020-01-02 22:38:21+00:00,2020-01-02 20:18:05+00:00,33.097,1,AG-1F01,1-1-193-825,4275.0,"[{'WhPerMile': 280, 'kWhRequested': 70.0, 'milesRequested': 250, 'minutesAvailable': 595, 'modifiedAt': 'Thu, 02 Jan 2020 13:37:11 GMT', 'paymentRequired': True, 'requestedDeparture': 'Thu, 02 Jan 2020 23:31:50 GMT', 'userID': 4275}]"
2,2020-01-02 13:56:35+00:00,2020-01-03 00:39:22+00:00,2020-01-02 16:35:06+00:00,6.521,1,AG-1F03,1-1-193-829,344.0,"[{'WhPerMile': 400, 'kWhRequested': 8.0, 'milesRequested': 20, 'minutesAvailable': 60, 'modifiedAt': 'Thu, 02 Jan 2020 13:57:17 GMT', 'paymentRequired': True, 'requestedDeparture': 'Thu, 02 Jan 2020 14:56:35 GMT', 'userID': 344}]"
3,2020-01-02 13:59:58+00:00,2020-01-02 16:38:39+00:00,2020-01-02 15:18:45+00:00,2.355,1,AG-1F04,1-1-193-820,1117.0,"[{'WhPerMile': 400, 'kWhRequested': 8.0, 'milesRequested': 20, 'minutesAvailable': 65, 'modifiedAt': 'Thu, 02 Jan 2020 14:00:03 GMT', 'paymentRequired': True, 'requestedDeparture': 'Thu, 02 Jan 2020 15:04:58 GMT', 'userID': 1117}, {'WhPerMile': 400, 'kWhRequested': 8.0, 'milesRequested': 20, 'minutesAvailable': 65, 'modifiedAt': 'Thu, 02 Jan 2020 14:00:19 GMT', 'paymentRequired': True, 'requestedDeparture': 'Thu, 02 Jan 2020 15:04:58 GMT', 'userID': 1117}]"
4,2020-01-02 14:00:01+00:00,2020-01-02 22:08:40+00:00,2020-01-02 18:17:30+00:00,13.375,1,AG-1F06,1-1-193-819,334.0,"[{'WhPerMile': 400, 'kWhRequested': 16.0, 'milesRequested': 40, 'minutesAvailable': 504, 'modifiedAt': 'Thu, 02 Jan 2020 14:00:13 GMT', 'paymentRequired': True, 'requestedDeparture': 'Thu, 02 Jan 2020 22:24:01 GMT', 'userID': 334}]"


In the last nunique() call, both spaceID and stationID have the same amount of unique data points. Since one identifies the parking space and the other the charging station, it is possible that every parking space has their own charging station, as opposed to one station servicing multiple spaces. If we can prove this, we could drop one of the two columns, since it is redundant.

In [19]:
#Create an aggregate column that contains the concatenated value of spaceID and stationID
charging_sessions_modified["combinedSpaceStationID"] = charging_sessions_modified["spaceID"].astype(str) \
                                                       + "/" + charging_sessions_modified["stationID"].astype(str)
charging_sessions_modified["combinedSpaceStationID"].nunique()

107

The combined column has the same amount of unique values, as the spaceID and stationID. Our assumption holds and we will thus drop the two columns and keep the aggregate. 

In [20]:
charging_sessions_modified.drop(columns = ["spaceID", "stationID"], inplace = True)

Convert the timestamp strings to datetime for easier conversion later on.

In [21]:
charging_sessions_modified["connectionTime"] = pd.to_datetime(charging_sessions_modified["connectionTime"])
charging_sessions_modified["disconnectTime"] = pd.to_datetime(charging_sessions_modified["disconnectTime"])
charging_sessions_modified["doneChargingTime"] = pd.to_datetime(charging_sessions_modified["doneChargingTime"],errors = "coerce")

In [22]:
charging_sessions_modified.head()

,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,siteID,userID,userInputs,combinedSpaceStationID
0,2020-01-02 13:08:54+00:00,2020-01-02 19:11:15+00:00,2020-01-02 17:31:35+00:00,25.016,1,194.0,"[{'WhPerMile': 250, 'kWhRequested': 25.0, 'milesRequested': 100, 'minutesAvailable': 463, 'modifiedAt': 'Thu, 02 Jan 2020 13:09:39 GMT', 'paymentRequired': True, 'requestedDeparture': 'Thu, 02 Jan 2020 20:51:54 GMT', 'userID': 194}]",AG-3F30/1-1-179-810
1,2020-01-02 13:36:50+00:00,2020-01-02 22:38:21+00:00,2020-01-02 20:18:05+00:00,33.097,1,4275.0,"[{'WhPerMile': 280, 'kWhRequested': 70.0, 'milesRequested': 250, 'minutesAvailable': 595, 'modifiedAt': 'Thu, 02 Jan 2020 13:37:11 GMT', 'paymentRequired': True, 'requestedDeparture': 'Thu, 02 Jan 2020 23:31:50 GMT', 'userID': 4275}]",AG-1F01/1-1-193-825
2,2020-01-02 13:56:35+00:00,2020-01-03 00:39:22+00:00,2020-01-02 16:35:06+00:00,6.521,1,344.0,"[{'WhPerMile': 400, 'kWhRequested': 8.0, 'milesRequested': 20, 'minutesAvailable': 60, 'modifiedAt': 'Thu, 02 Jan 2020 13:57:17 GMT', 'paymentRequired': True, 'requestedDeparture': 'Thu, 02 Jan 2020 14:56:35 GMT', 'userID': 344}]",AG-1F03/1-1-193-829
3,2020-01-02 13:59:58+00:00,2020-01-02 16:38:39+00:00,2020-01-02 15:18:45+00:00,2.355,1,1117.0,"[{'WhPerMile': 400, 'kWhRequested': 8.0, 'milesRequested': 20, 'minutesAvailable': 65, 'modifiedAt': 'Thu, 02 Jan 2020 14:00:03 GMT', 'paymentRequired': True, 'requestedDeparture': 'Thu, 02 Jan 2020 15:04:58 GMT', 'userID': 1117}, {'WhPerMile': 400, 'kWhRequested': 8.0, 'milesRequested': 20, 'minutesAvailable': 65, 'modifiedAt': 'Thu, 02 Jan 2020 14:00:19 GMT', 'paymentRequired': True, 'requestedDeparture': 'Thu, 02 Jan 2020 15:04:58 GMT', 'userID': 1117}]",AG-1F04/1-1-193-820
4,2020-01-02 14:00:01+00:00,2020-01-02 22:08:40+00:00,2020-01-02 18:17:30+00:00,13.375,1,334.0,"[{'WhPerMile': 400, 'kWhRequested': 16.0, 'milesRequested': 40, 'minutesAvailable': 504, 'modifiedAt': 'Thu, 02 Jan 2020 14:00:13 GMT', 'paymentRequired': True, 'requestedDeparture': 'Thu, 02 Jan 2020 22:24:01 GMT', 'userID': 334}]",AG-1F06/1-1-193-819


Let's do a check, whether we have any data where the disconnectTime occurs before the connectionTime. Since this is logically impossible, we would drop this data.

In [23]:
charging_sessions_modified[charging_sessions_modified["disconnectTime"]<charging_sessions_modified["connectionTime"]]

,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,siteID,userID,userInputs,combinedSpaceStationID


Doesn't occur anywhere, we're good on this front.  
At this point we are gonna make an assumption on the dataset. We still have null values in the doneChargingTime column. Our assumption is, these values are null if disconnect and doneChargingTime would be the same, i.e. if the the vehicle gets disconnected before it finished charging. We try to generate some supporting evidence, by checking if the disconnect and doneCharging are the same anywhere in the dataset.

In [24]:
charging_sessions_modified[charging_sessions_modified["disconnectTime"] == charging_sessions_modified["doneChargingTime"]]

,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,siteID,userID,userInputs,combinedSpaceStationID
25051,2019-05-31 22:26:39+00:00,2019-06-01 00:24:47+00:00,2019-06-01 00:24:47+00:00,5.933000,2,431.0,"[{'WhPerMile': 300, 'kWhRequested': 18.0, 'milesRequested': 60, 'minutesAvailable': 149, 'modifiedAt': 'Fri, 31 May 2019 22:27:14 GMT', 'paymentRequired': True, 'requestedDeparture': 'Sat, 01 Jun 2019 00:55:39 GMT', 'userID': 431}]",CA-322/2-39-78-360
35096,2018-07-09 00:21:39+00:00,2018-07-09 06:16:09+00:00,2018-07-09 06:16:09+00:00,27.154000,2,NaN,NaN,CA-320/2-39-78-363
40658,2018-09-20 18:21:06+00:00,2018-09-20 20:20:04+00:00,2018-09-20 20:20:04+00:00,3.584000,2,NaN,NaN,CA-323/2-39-78-366
41408,2018-09-29 22:03:35+00:00,2018-09-29 23:00:39+00:00,2018-09-29 23:00:39+00:00,6.815959,2,NaN,NaN,CA-323/2-39-78-366
42038,2018-10-08 00:36:11+00:00,2018-10-08 01:42:18+00:00,2018-10-08 01:42:18+00:00,7.191020,2,NaN,NaN,CA-323/2-39-78-366


Because we have a few data points, where the disconnect and doneCharging time are identical, we don't have any evidence to support our earlier assumption. It might still be possible that it holds true, but we will play it safe and drop all the rows where the doneChargingTime is null.

In [25]:
charging_sessions_modified.dropna(axis = 0, subset = "doneChargingTime", inplace=True)

In [26]:
charging_sessions_modified.info()

<class 'pandas.core.frame.DataFrame'>
Index: 60950 entries, 0 to 65036
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype              
---  ------                  --------------  -----              
 0   connectionTime          60950 non-null  datetime64[ns, UTC]
 1   disconnectTime          60950 non-null  datetime64[ns, UTC]
 2   doneChargingTime        60950 non-null  datetime64[ns, UTC]
 3   kWhDelivered            60950 non-null  float64            
 4   siteID                  60950 non-null  int64              
 5   userID                  44644 non-null  float64            
 6   userInputs              44644 non-null  object             
 7   combinedSpaceStationID  60950 non-null  object             
dtypes: datetime64[ns, UTC](3), float64(2), int64(1), object(2)
memory usage: 4.2+ MB


This leaves us with null-values only in the userID and inputs column. We will deal with these later on.
Before that, we will add a few columns that contain relevant metrics. Starting with the total time the vehicle was connected to the charging station, the time it was actually charging and the idle time (each in seconds).

In [27]:
charging_sessions_modified = charging_sessions_modified.assign(timeConnected = lambda x : ((x["disconnectTime"]-x["connectionTime"])))
charging_sessions_modified["timeConnected"] = charging_sessions_modified["timeConnected"].dt.total_seconds()

charging_sessions_modified = charging_sessions_modified.assign(idleTime = lambda x : ((x["disconnectTime"]-x["doneChargingTime"])))
charging_sessions_modified["idleTime"] = charging_sessions_modified["idleTime"].dt.total_seconds()

charging_sessions_modified = charging_sessions_modified.assign(chargingTime = lambda x : ((x["doneChargingTime"]-x["connectionTime"])))
charging_sessions_modified["chargingTime"] = charging_sessions_modified["chargingTime"].dt.total_seconds()

In [28]:
charging_sessions_modified.head()

,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,siteID,userID,userInputs,combinedSpaceStationID,timeConnected,idleTime,chargingTime
0,2020-01-02 13:08:54+00:00,2020-01-02 19:11:15+00:00,2020-01-02 17:31:35+00:00,25.016,1,194.0,"[{'WhPerMile': 250, 'kWhRequested': 25.0, 'milesRequested': 100, 'minutesAvailable': 463, 'modifiedAt': 'Thu, 02 Jan 2020 13:09:39 GMT', 'paymentRequired': True, 'requestedDeparture': 'Thu, 02 Jan 2020 20:51:54 GMT', 'userID': 194}]",AG-3F30/1-1-179-810,21741.0,5980.0,15761.0
1,2020-01-02 13:36:50+00:00,2020-01-02 22:38:21+00:00,2020-01-02 20:18:05+00:00,33.097,1,4275.0,"[{'WhPerMile': 280, 'kWhRequested': 70.0, 'milesRequested': 250, 'minutesAvailable': 595, 'modifiedAt': 'Thu, 02 Jan 2020 13:37:11 GMT', 'paymentRequired': True, 'requestedDeparture': 'Thu, 02 Jan 2020 23:31:50 GMT', 'userID': 4275}]",AG-1F01/1-1-193-825,32491.0,8416.0,24075.0
2,2020-01-02 13:56:35+00:00,2020-01-03 00:39:22+00:00,2020-01-02 16:35:06+00:00,6.521,1,344.0,"[{'WhPerMile': 400, 'kWhRequested': 8.0, 'milesRequested': 20, 'minutesAvailable': 60, 'modifiedAt': 'Thu, 02 Jan 2020 13:57:17 GMT', 'paymentRequired': True, 'requestedDeparture': 'Thu, 02 Jan 2020 14:56:35 GMT', 'userID': 344}]",AG-1F03/1-1-193-829,38567.0,29056.0,9511.0
3,2020-01-02 13:59:58+00:00,2020-01-02 16:38:39+00:00,2020-01-02 15:18:45+00:00,2.355,1,1117.0,"[{'WhPerMile': 400, 'kWhRequested': 8.0, 'milesRequested': 20, 'minutesAvailable': 65, 'modifiedAt': 'Thu, 02 Jan 2020 14:00:03 GMT', 'paymentRequired': True, 'requestedDeparture': 'Thu, 02 Jan 2020 15:04:58 GMT', 'userID': 1117}, {'WhPerMile': 400, 'kWhRequested': 8.0, 'milesRequested': 20, 'minutesAvailable': 65, 'modifiedAt': 'Thu, 02 Jan 2020 14:00:19 GMT', 'paymentRequired': True, 'requestedDeparture': 'Thu, 02 Jan 2020 15:04:58 GMT', 'userID': 1117}]",AG-1F04/1-1-193-820,9521.0,4794.0,4727.0
4,2020-01-02 14:00:01+00:00,2020-01-02 22:08:40+00:00,2020-01-02 18:17:30+00:00,13.375,1,334.0,"[{'WhPerMile': 400, 'kWhRequested': 16.0, 'milesRequested': 40, 'minutesAvailable': 504, 'modifiedAt': 'Thu, 02 Jan 2020 14:00:13 GMT', 'paymentRequired': True, 'requestedDeparture': 'Thu, 02 Jan 2020 22:24:01 GMT', 'userID': 334}]",AG-1F06/1-1-193-819,29319.0,13870.0,15449.0


Other useful columns that we will now add for seasonality and temporal pattern analysis are the day of the week, the time and the month

In [29]:
charging_sessions_modified["weekday"] = charging_sessions_modified["connectionTime"].dt.day_of_week
charging_sessions_modified["month"] = charging_sessions_modified["connectionTime"].dt.month
charging_sessions_modified["time"] = charging_sessions_modified["connectionTime"].dt.time
charging_sessions_modified.head()

,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,siteID,userID,userInputs,combinedSpaceStationID,timeConnected,idleTime,chargingTime,weekday,month,time
0,2020-01-02 13:08:54+00:00,2020-01-02 19:11:15+00:00,2020-01-02 17:31:35+00:00,25.016,1,194.0,"[{'WhPerMile': 250, 'kWhRequested': 25.0, 'milesRequested': 100, 'minutesAvailable': 463, 'modifiedAt': 'Thu, 02 Jan 2020 13:09:39 GMT', 'paymentRequired': True, 'requestedDeparture': 'Thu, 02 Jan 2020 20:51:54 GMT', 'userID': 194}]",AG-3F30/1-1-179-810,21741.0,5980.0,15761.0,3,1,13:08:54
1,2020-01-02 13:36:50+00:00,2020-01-02 22:38:21+00:00,2020-01-02 20:18:05+00:00,33.097,1,4275.0,"[{'WhPerMile': 280, 'kWhRequested': 70.0, 'milesRequested': 250, 'minutesAvailable': 595, 'modifiedAt': 'Thu, 02 Jan 2020 13:37:11 GMT', 'paymentRequired': True, 'requestedDeparture': 'Thu, 02 Jan 2020 23:31:50 GMT', 'userID': 4275}]",AG-1F01/1-1-193-825,32491.0,8416.0,24075.0,3,1,13:36:50
2,2020-01-02 13:56:35+00:00,2020-01-03 00:39:22+00:00,2020-01-02 16:35:06+00:00,6.521,1,344.0,"[{'WhPerMile': 400, 'kWhRequested': 8.0, 'milesRequested': 20, 'minutesAvailable': 60, 'modifiedAt': 'Thu, 02 Jan 2020 13:57:17 GMT', 'paymentRequired': True, 'requestedDeparture': 'Thu, 02 Jan 2020 14:56:35 GMT', 'userID': 344}]",AG-1F03/1-1-193-829,38567.0,29056.0,9511.0,3,1,13:56:35
3,2020-01-02 13:59:58+00:00,2020-01-02 16:38:39+00:00,2020-01-02 15:18:45+00:00,2.355,1,1117.0,"[{'WhPerMile': 400, 'kWhRequested': 8.0, 'milesRequested': 20, 'minutesAvailable': 65, 'modifiedAt': 'Thu, 02 Jan 2020 14:00:03 GMT', 'paymentRequired': True, 'requestedDeparture': 'Thu, 02 Jan 2020 15:04:58 GMT', 'userID': 1117}, {'WhPerMile': 400, 'kWhRequested': 8.0, 'milesRequested': 20, 'minutesAvailable': 65, 'modifiedAt': 'Thu, 02 Jan 2020 14:00:19 GMT', 'paymentRequired': True, 'requestedDeparture': 'Thu, 02 Jan 2020 15:04:58 GMT', 'userID': 1117}]",AG-1F04/1-1-193-820,9521.0,4794.0,4727.0,3,1,13:59:58
4,2020-01-02 14:00:01+00:00,2020-01-02 22:08:40+00:00,2020-01-02 18:17:30+00:00,13.375,1,334.0,"[{'WhPerMile': 400, 'kWhRequested': 16.0, 'milesRequested': 40, 'minutesAvailable': 504, 'modifiedAt': 'Thu, 02 Jan 2020 14:00:13 GMT', 'paymentRequired': True, 'requestedDeparture': 'Thu, 02 Jan 2020 22:24:01 GMT', 'userID': 334}]",AG-1F06/1-1-193-819,29319.0,13870.0,15449.0,3,1,14:00:01


Now we will check for erroneous data points.

In [30]:
charging_sessions_modified.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
kWhDelivered,60950.0,11.583268,10.153601,0.501,5.013,8.991,13.984,108.797242
siteID,60950.0,1.481674,0.499668,1.000,1.000,1.000,2.000,2.000000
userID,44644.0,1709.660514,2464.480169,1.000,431.000,774.000,1754.000,19923.000000
timeConnected,60950.0,22960.401198,17717.646112,183.000,10708.500,22705.000,33267.000,882969.000000
idleTime,60950.0,10271.909319,14447.964854,-3596.000,13.000,5774.000,17712.750,562037.000000
chargingTime,60950.0,12688.491879,11032.329226,-2482.000,5823.250,9933.000,17011.000,720057.000000
weekday,60950.0,2.281920,1.650711,0.000,1.000,2.000,4.000,6.000000
month,60950.0,6.596702,3.370832,1.000,4.000,7.000,9.000,12.000000


There are definitely a few outliers we need to take care off. Negative values in idle and charging time make no sense logically, we will evaluate these to see if its erroneous data or another error we might have introduced.

In [31]:
charging_sessions_modified[charging_sessions_modified["idleTime"]<0]

,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,siteID,userID,userInputs,combinedSpaceStationID,timeConnected,idleTime,chargingTime,weekday,month,time
12,2020-01-02 15:04:38+00:00,2020-01-02 22:08:39+00:00,2020-01-02 22:09:36+00:00,25.567,1,528.0,"[{'WhPerMile': 250, 'kWhRequested': 50.0, 'milesRequested': 200, 'minutesAvailable': 555, 'modifiedAt': 'Thu, 02 Jan 2020 15:04:49 GMT', 'paymentRequired': True, 'requestedDeparture': 'Fri, 03 Jan 2020 00:19:38 GMT', 'userID': 528}]",AG-1F07/1-1-178-824,25441.0,-57.0,25498.0,3,1,15:04:38
20,2020-01-02 15:28:47+00:00,2020-01-02 19:01:54+00:00,2020-01-02 19:02:51+00:00,7.417,1,1283.0,"[{'WhPerMile': 350, 'kWhRequested': 42.0, 'milesRequested': 120, 'minutesAvailable': 238, 'modifiedAt': 'Thu, 02 Jan 2020 15:29:16 GMT', 'paymentRequired': True, 'requestedDeparture': 'Thu, 02 Jan 2020 19:26:47 GMT', 'userID': 1283}, {'WhPerMile': 350, 'kWhRequested': 56.0, 'milesRequested': 160, 'minutesAvailable': 282, 'modifiedAt': 'Thu, 02 Jan 2020 16:29:35 GMT', 'paymentRequired': True, 'requestedDeparture': 'Thu, 02 Jan 2020 20:10:47 GMT', 'userID': 1283}]",AG-1F02/1-1-193-827,12787.0,-57.0,12844.0,3,1,15:28:47
25,2020-01-02 15:42:05+00:00,2020-01-02 21:58:45+00:00,2020-01-02 21:59:42+00:00,36.701,1,474.0,"[{'WhPerMile': 400, 'kWhRequested': 32.0, 'milesRequested': 80, 'minutesAvailable': 532, 'modifiedAt': 'Thu, 02 Jan 2020 15:42:11 GMT', 'paymentRequired': True, 'requestedDeparture': 'Fri, 03 Jan 2020 00:34:05 GMT', 'userID': 474}]",AG-3F23/1-1-179-797,22600.0,-57.0,22657.0,3,1,15:42:05
26,2020-01-02 15:57:24+00:00,2020-01-02 16:35:37+00:00,2020-01-02 16:36:34+00:00,3.689,1,724.0,"[{'WhPerMile': 400, 'kWhRequested': 8.0, 'milesRequested': 20, 'minutesAvailable': 89, 'modifiedAt': 'Thu, 02 Jan 2020 15:57:39 GMT', 'paymentRequired': True, 'requestedDeparture': 'Thu, 02 Jan 2020 17:26:24 GMT', 'userID': 724}]",AG-3F31/1-1-179-781,2293.0,-57.0,2350.0,3,1,15:57:24
33,2020-01-02 16:34:35+00:00,2020-01-02 18:49:41+00:00,2020-01-02 18:50:38+00:00,7.120,1,2276.0,"[{'WhPerMile': 600, 'kWhRequested': 18.0, 'milesRequested': 30, 'minutesAvailable': 165, 'modifiedAt': 'Thu, 02 Jan 2020 16:35:06 GMT', 'paymentRequired': True, 'requestedDeparture': 'Thu, 02 Jan 2020 19:19:35 GMT', 'userID': 2276}]",AG-3F19/1-1-179-790,8106.0,-57.0,8163.0,3,1,16:34:35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65028,2019-07-01 19:20:31+00:00,2019-07-02 00:16:32+00:00,2019-07-02 00:16:42+00:00,26.324,1,458.0,"[{'WhPerMile': 400, 'kWhRequested': 8.0, 'milesRequested': 20, 'minutesAvailable': 204, 'modifiedAt': 'Mon, 01 Jul 2019 19:23:09 GMT', 'paymentRequired': True, 'requestedDeparture': 'Mon, 01 Jul 2019 22:44:31 GMT', 'userID': 458}, {'WhPerMile': 400, 'kWhRequested': 84.0, 'milesRequested': 210, 'minutesAvailable': 228, 'modifiedAt': 'Mon, 01 Jul 2019 19:25:18 GMT', 'paymentRequired': True, 'requestedDeparture': 'Mon, 01 Jul 2019 23:08:31 GMT', 'userID': 458}, {'WhPerMile': 400, 'kWhRequested': 68.0, 'milesRequested': 170, 'minutesAvailable': 180, 'modifiedAt': 'Mon, 01 Jul 2019 19:26:15 GMT', 'paymentRequired': True, 'requestedDeparture': 'Mon, 01 Jul 2019 22:20:31 GMT', 'userID': 458}]",AG-3F29/1-1-179-783,17761.0,-10.0,17771.0,0,7,19:20:31
65030,2019-07-01 20:01:07+00:00,2019-07-02 00:32:26+00:00,2019-07-02 00:32:59+00:00,21.588,1,1479.0,"[{'WhPerMile': 275, 'kWhRequested': 19.25, 'milesRequested': 70, 'minutesAvailable': 286, 'modifiedAt': 'Mon, 01 Jul 2019 20:03:21 GMT', 'paymentRequired': True, 'requestedDeparture': 'Tue, 02 Jul 2019 00:47:07 GMT', 'userID': 1479}, {'WhPerMile': 200, 'kWhRequested': 30.0, 'milesRequested': 150, 'minutesAvailable': 286, 'modifiedAt': 'Mon, 01 Jul 2019 23:51:42 GMT', 'paymentRequired': True, 'requestedDeparture': 'Tue, 02 Jul 2019 00:47:07 GMT', 'userID': 1479}, {'WhPerMile': 200, 'kWhRequested': 30.0, 'milesRequested': 150, 'minutesAvailable': 288, 'modifiedAt': 'Mon, 01 Jul 2019 23:59:01 GMT', 'paymentRequired': True, 'requestedDeparture': 'Tue, 02 Jul 2019 0

In [32]:
charging_sessions_modified[charging_sessions_modified["idleTime"]<0].describe().transpose()

,count,mean,std,min,25%,50%,75%,max
kWhDelivered,4387.0,14.239556,12.575748,0.508,5.379,11.161,19.2395,108.797242
siteID,4387.0,1.268977,0.443478,1.000,1.000,1.000,2.0000,2.000000
userID,3329.0,1458.393812,1532.613614,22.000,410.000,832.000,2052.0000,17097.000000
timeConnected,4387.0,15629.237520,14617.364141,242.000,6534.500,12291.000,21362.5000,540779.000000
idleTime,4387.0,-56.017096,76.290027,-3596.000,-59.000,-58.000,-57.0000,-1.000000
chargingTime,4387.0,15685.254616,14615.972474,253.000,6592.500,12351.000,21417.0000,540838.000000
weekday,4387.0,2.247322,1.660041,0.000,1.000,2.000,4.0000,6.000000
month,4387.0,7.178938,3.401890,1.000,5.000,8.000,10.0000,12.000000


The mean of the negative idle times is around a minute, with most values being around this range. This could indicate a situation where the station does not stop charging or does not register that it stopped charging for around a minute after the car was disconnected. 
Because we have no further information on this, we will take the safe route and drop them.

In [33]:
charging_sessions_modified.drop(charging_sessions_modified[charging_sessions_modified.idleTime<0].index, inplace = True)

Another potential problem are the max values for timeConnected, idleTime and chargingTime. While it is possible to be connected and charging for the up to 10 days the max values represent, it is not that realistic. Let's have a closer look at all data points, where the connection time is above 24 hours (86400 seconds)

In [34]:
charging_sessions_modified[charging_sessions_modified["timeConnected"]>=86400]

,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,siteID,userID,userInputs,combinedSpaceStationID,timeConnected,idleTime,chargingTime,weekday,month,time
7627,2019-08-14 13:00:40+00:00,2019-08-16 02:24:39+00:00,2019-08-14 14:48:08+00:00,4.557,1,521.0,"[{'WhPerMile': 400, 'kWhRequested': 16.0, 'milesRequested': 40, 'minutesAvailable': 73, 'modifiedAt': 'Wed, 14 Aug 2019 13:00:41 GMT', 'paymentRequired': True, 'requestedDeparture': 'Wed, 14 Aug 2019 14:13:40 GMT', 'userID': 521}]",AG-1F12/1-1-194-822,134639.0,128191.0,6448.0,2,8,13:00:40
7911,2019-08-20 14:20:30+00:00,2019-08-22 00:32:21+00:00,2019-08-20 17:55:00+00:00,14.022,1,2187.0,"[{'WhPerMile': 333, 'kWhRequested': 19.98, 'milesRequested': 60, 'minutesAvailable': 212, 'modifiedAt': 'Tue, 20 Aug 2019 14:21:02 GMT', 'paymentRequired': True, 'requestedDeparture': 'Tue, 20 Aug 2019 17:52:30 GMT', 'userID': 2187}]",AG-3F28/1-1-179-777,123111.0,110241.0,12870.0,1,8,14:20:30
18601,2021-07-16 14:01:18+00:00,2021-07-19 23:05:55+00:00,2021-07-16 18:16:51+00:00,11.286,1,382.0,"[{'WhPerMile': 350, 'kWhRequested': 17.5, 'milesRequested': 50, 'minutesAvailable': 240, 'modifiedAt': 'Fri, 16 Jul 2021 14:01:17 GMT', 'paymentRequired': True, 'requestedDeparture': 'Fri, 16 Jul 2021 18:01:18 GMT', 'userID': 382}]",AG-1F10/1-1-178-828,291877.0,276544.0,15333.0,4,7,14:01:18
18603,2021-07-16 15:39:02+00:00,2021-07-19 23:05:38+00:00,2021-07-16 18:16:54+00:00,13.377,1,10747.0,"[{'WhPerMile': 283, 'kWhRequested': 28.3, 'milesRequested': 100, 'minutesAvailable': 201, 'modifiedAt': 'Fri, 16 Jul 2021 15:39:02 GMT', 'paymentRequired': True, 'requestedDeparture': 'Fri, 16 Jul 2021 19:00:02 GMT', 'userID': 10747}]",AG-1F08/1-1-178-823,285996.0,276524.0,9472.0,4,7,15:39:02
18604,2021-07-16 15:45:30+00:00,2021-07-19 23:06:05+00:00,2021-07-19 20:36:25+00:00,5.793,1,3161.0,"[{'WhPerMile': 250, 'kWhRequested': 8.75, 'milesRequested': 35, 'minutesAvailable': 150, 'modifiedAt': 'Fri, 16 Jul 2021 15:45:29 GMT', 'paymentRequired': True, 'requestedDeparture': 'Fri, 16 Jul 2021 18:15:30 GMT', 'userID': 3161}]",AG-3F29/1-1-179-783,285635.0,8980.0,276655.0,4,7,15:45:30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54521,2018-11-12 19:35:55+00:00,2018-11-15 01:44:47+00:00,2018-11-12 23:41:28+00:00,10.646,1,425.0,"[{'WhPerMile': 300, 'kWhRequested': 12.0, 'milesRequested': 40, 'minutesAvailable': 334, 'modifiedAt': 'Mon, 12 Nov 2018 19:35:56 GMT', 'paymentRequired': True, 'requestedDeparture': 'Tue, 13 Nov 2018 01:09:55 GMT', 'userID': 425}]",AG-4F43/1-1-191-778,194932.0,180199.0,14733.0,0,11,19:35:55
61169,2019-04-16 00:01:28+00:00,2019-04-18 00:20:42+00:00,2019-04-17 01:18:18+00:00,39.970,1,2050.0,"[{'WhPerMile': 600, 'kWhRequested': 12.0, 'milesRequested': 20, 'minutesAvailable': 244, 'modifiedAt': 'Tue, 16 Apr 2019 00:03:15 GMT', 'paymentRequired': True, 'requestedDeparture': 'Tue, 16 Apr 2019 04:05:28 GMT', 'userID': 2050}, {'WhPerMile': 333, 'kWhRequested': 23.31, 'milesRequested': 70, 'minutesAvailable': 244, 'modifiedAt': 'Tue, 16 Apr 2019 02:40:44 GMT', 'paymentRequired': True, 'requestedDeparture': 'Tue, 16 Apr 2019 04:05:28 GMT', 'userID': 2050}, {'WhPerMile': 333, 'kWhRequested': 39.96, 'milesRequested': 120, 'minutesAvailable': 244, 'modifiedAt': 'Tue, 16 Apr 2019 02:41:46 GMT', 'paymentRequired': True, 'requestedDeparture': 'Tue, 16 Apr 2019 04:05:28 GMT', 'userID': 2050}, {'WhPerMile': 333, 'kWhRequested': 39.96, 'milesRequested': 120, 'minutesAvailable': 878, 'modifiedAt': 'Tue, 16 Apr 2019 02:42:02 GMT', 'paymentRequired': True, 'requestedDeparture': 'Tue, 16 Apr 2019 14:39:28 GMT', 'userID': 2050}]",AG-4F37/1-1-191-792,173954.0,82944.0,91010.0,1,4,00:01:28
62402,2019-05-09 00:09:47+00:00,2019-05-10 01:23:16+00:00,2019-05-09 06:04:05+00:00,36.682,1,2050.0,"[{'WhPerMile': 333, 'kWhRequested': 39.96, 'milesRequested': 120, 'minutesAvailable': 720, 'modifiedAt': 'Thu, 09 May 2019 00:10:02 GMT', 'paymentRequired': True, 'requestedDeparture': 'Thu, 09 May 2019 12:09:47 GMT', 'userID'

In [35]:
charging_sessions_modified[charging_sessions_modified["timeConnected"]>=86400].describe().transpose()

,count,mean,std,min,25%,50%,75%,max
kWhDelivered,266.0,15.758231,13.145946,0.855,6.5965,13.1915,15.909,61.114
siteID,266.0,1.932331,0.251651,1.000,2.0000,2.0000,2.000,2.000
userID,122.0,1844.532787,2382.089881,67.000,565.0000,1153.0000,1826.000,15803.000
timeConnected,266.0,169737.744361,103165.204918,86495.000,101506.0000,134621.0000,198945.250,882969.000
idleTime,266.0,120326.748120,93859.917000,4.000,69610.5000,93952.5000,152027.250,562037.000
chargingTime,266.0,49410.996241,78455.038387,-59.000,7995.7500,16653.0000,71447.250,720057.000
weekday,266.0,3.191729,2.047752,0.000,1.0000,4.0000,5.000,6.000
month,266.0,6.815789,2.896956,1.000,5.0000,7.0000,9.000,12.000


271 data points where the connection time is more than a day is quite a lot. The mean of these being around 2 days is also higher than we would have expected. Nevertheless, we will leave these in the dataset. While it looks rather strange to have your car charging for more than 2 days on a charger than is not on your private property, this isn't straight up impossible like the negative idle times.

In [36]:
charging_sessions_modified.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
kWhDelivered,56563.0,11.377248,9.911543,0.501,4.997,8.834,13.911,75.528
siteID,56563.0,1.498170,0.500001,1.000,1.000,1.000,2.000,2.000
userID,41315.0,1729.906596,2523.560134,1.000,438.000,766.000,1746.000,19923.000
timeConnected,56563.0,23529.002846,17810.170154,183.000,11339.500,24003.000,33511.500,882969.000
idleTime,56563.0,11072.938493,14697.578410,0.000,505.000,7253.000,18644.000,562037.000
chargingTime,56563.0,12456.064353,10669.420682,-2482.000,5763.000,9783.000,16735.000,720057.000
weekday,56563.0,2.284603,1.649970,0.000,1.000,2.000,4.000,6.000
month,56563.0,6.551544,3.364233,1.000,4.000,7.000,9.000,12.000


Last but not least, we have the negative charging time to deal with. We will probably deal with these, the same way as the idle time by removing them. Let's have a look at them first though.

In [37]:
charging_sessions_modified[charging_sessions_modified.chargingTime<0].describe().transpose()

,count,mean,std,min,25%,50%,75%,max
kWhDelivered,27.0,2.479419,3.555755,0.521,0.815942,1.06723,1.707816,14.967
siteID,27.0,1.962963,0.192450,1.000,2.000000,2.00000,2.000000,2.000
userID,6.0,889.500000,421.951538,483.000,527.000000,857.00000,1154.000000,1470.000
timeConnected,27.0,32152.222222,50699.951537,1123.000,12954.000000,19344.00000,26277.500000,255592.000
idleTime,27.0,32448.851852,50554.202093,3605.000,13012.500000,19400.00000,26335.000000,255650.000
chargingTime,27.0,-296.629630,707.666868,-2482.000,-60.000000,-59.00000,-57.500000,-1.000
weekday,27.0,2.666667,1.818706,0.000,1.000000,3.00000,4.000000,6.000
month,27.0,7.851852,2.878350,3.000,5.000000,9.00000,10.000000,11.000


The minimum of them being around 40 minutes is quite interesting. There could be a few reasons for these values:
* corrupt data
* misfunction of the charging station
* mishandling of time-zones

Whatever the reason, this is not data we are able to work with, so we will drop them.

In [38]:
charging_sessions_modified.drop(charging_sessions_modified[charging_sessions_modified.chargingTime<0].index, inplace = True)

Now let's check the userInputs column. Since we have null values over more than a fourth of the data-set there, simply dropping them might not be the best solution. Let's have a closer look at the data to see if they are of any use to us.

In [39]:
charging_sessions_modified.info()

<class 'pandas.core.frame.DataFrame'>
Index: 56536 entries, 0 to 65036
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype              
---  ------                  --------------  -----              
 0   connectionTime          56536 non-null  datetime64[ns, UTC]
 1   disconnectTime          56536 non-null  datetime64[ns, UTC]
 2   doneChargingTime        56536 non-null  datetime64[ns, UTC]
 3   kWhDelivered            56536 non-null  float64            
 4   siteID                  56536 non-null  int64              
 5   userID                  41309 non-null  float64            
 6   userInputs              41309 non-null  object             
 7   combinedSpaceStationID  56536 non-null  object             
 8   timeConnected           56536 non-null  float64            
 9   idleTime                56536 non-null  float64            
 10  chargingTime            56536 non-null  float64            
 11  weekday                 56536 non-null  int32 

In [40]:
userInputs = pd.DataFrame(charging_sessions_modified.copy()["userInputs"])
userInputs

,userInputs
0,"[{'WhPerMile': 250, 'kWhRequested': 25.0, 'milesRequested': 100, 'minutesAvailable': 463, 'modifiedAt': 'Thu, 02 Jan 2020 13:09:39 GMT', 'paymentRequired': True, 'requestedDeparture': 'Thu, 02 Jan 2020 20:51:54 GMT', 'userID': 194}]"
1,"[{'WhPerMile': 280, 'kWhRequested': 70.0, 'milesRequested': 250, 'minutesAvailable': 595, 'modifiedAt': 'Thu, 02 Jan 2020 13:37:11 GMT', 'paymentRequired': True, 'requestedDeparture': 'Thu, 02 Jan 2020 23:31:50 GMT', 'userID': 4275}]"
2,"[{'WhPerMile': 400, 'kWhRequested': 8.0, 'milesRequested': 20, 'minutesAvailable': 60, 'modifiedAt': 'Thu, 02 Jan 2020 13:57:17 GMT', 'paymentRequired': True, 'requestedDeparture': 'Thu, 02 Jan 2020 14:56:35 GMT', 'userID': 344}]"
3,"[{'WhPerMile': 400, 'kWhRequested': 8.0, 'milesRequested': 20, 'minutesAvailable': 65, 'modifiedAt': 'Thu, 02 Jan 2020 14:00:03 GMT', 'paymentRequired': True, 'requestedDeparture': 'Thu, 02 Jan 2020 15:04:58 GMT', 'userID': 1117}, {'WhPerMile': 400, 'kWhRequested': 8.0, 'milesRequested': 20, 'minutesAvailable': 65, 'modifiedAt': 'Thu, 02 Jan 2020 14:00:19 GMT', 'paymentRequired': True, 'requestedDeparture': 'Thu, 02 Jan 2020 15:04:58 GMT', 'userID': 1117}]"
4,"[{'WhPerMile': 400, 'kWhRequested': 16.0, 'milesRequested': 40, 'minutesAvailable': 504, 'modifiedAt': 'Thu, 02 Jan 2020 14:00:13 GMT', 'paymentRequired': True, 'requestedDeparture': 'Thu, 02 Jan 2020 22:24:01 GMT', 'userID': 334}]"
...,...
65025,"[{'WhPerMile': 275, 'kWhRequested': 8.25, 'milesRequested': 30, 'minutesAvailable': 374, 'modifiedAt': 'Mon, 01 Jul 2019 19:13:10 GMT', 'paymentRequired': True, 'requestedDeparture': 'Tue, 02 Jul 2019 01:27:11 GMT', 'userID': 2643}]"
65029,"[{'WhPerMile': 231, 'kWhRequested': 36.96, 'milesRequested': 160, 'minutesAvailable': 310, 'modifiedAt': 'Mon, 01 Jul 2019 19:43:58 GMT', 'paymentRequired': True, 'requestedDeparture': 'Tue, 02 Jul 2019 00:53:46 GMT', 'userID': 394}, {'WhPerMile': 231, 'kWhRequested': 43.89, 'milesRequested': 190, 'minutesAvailable': 443, 'modifiedAt': 'Mon, 01 Jul 2019 20:30:28 GMT', 'paymentRequired': True, 'requestedDeparture': 'Tue, 02 Jul 2019 03:06:46 GMT', 'userID': 394}]"
65031,"[{'WhPerMile': 300, 'kWhRequested': 12.0, 'milesRequested': 40, 'minutesAvailable': 381, 'modifiedAt': 'Mon, 01 Jul 2019 20:34:29 GMT', 'paymentRequired': True, 'requestedDeparture': 'Tue, 02 Jul 2019 02:55:30 GMT', 'userID': 2276}]"
65032,"[{'WhPerMile': 400, 'kWhRequested': 32.0, 'milesRequested': 80, 'minutesAvailable': 36, 'modifiedAt': 'Mon, 01 Jul 2019 21:49:37 GMT', 'paymentRequired': True, 'requestedDeparture': 'Mon, 01 Jul 2019 22:25:12 GMT', 'userID': 1346}, {'WhPerMile': 400, 'kWhRequested': 32.0, 'milesRequested': 80, 'minutesAvailable': 36, 'modifiedAt': 'Mon, 01 Jul 2019 21:51:07 GMT', 'paymentRequired': True, 'requestedDeparture': 'Mon, 01 Jul 2019 22:25:12 GMT', 'userID': 1346}]"


In [41]:
import numpy as np
# Convert string representations of dictionaries to actual dictionaries
userInputs['userInputs'] = userInputs['userInputs'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) and pd.notnull(x) else x)

# Define the default dictionary for filling None values within each dictionary
default_dict = {'WhPerMile': np.NaN, 'kWhRequested': np.NaN, 'milesRequested': np.NaN,
                'minutesAvailable': np.NaN, 'modifiedAt':  np.NaN, 'paymentRequired': None,
                'requestedDeparture': np.NaN}

# Function to fill None values within a dictionary with the default dictionary
def fill_none_values(entry):
    if isinstance(entry, dict):
        return {key: default_dict[key] if entry.get(key) is None else entry[key] for key in default_dict}
    elif isinstance(entry, list) and entry and isinstance(entry[0], dict):
        return entry[0]
    else:
        return default_dict

# Apply the fill_none_values function to the "userInputs" column
userInputs['userInputs'] = userInputs['userInputs'].apply(fill_none_values)

# Use json_normalize to flatten the nested dictionaries
user_df = json_normalize(userInputs['userInputs'])

In [42]:
user_df

,WhPerMile,kWhRequested,milesRequested,minutesAvailable,modifiedAt,paymentRequired,requestedDeparture,userID
0,250.0,25.00,100.0,463.0,"Thu, 02 Jan 2020 13:09:39 GMT",True,"Thu, 02 Jan 2020 20:51:54 GMT",194.0
1,280.0,70.00,250.0,595.0,"Thu, 02 Jan 2020 13:37:11 GMT",True,"Thu, 02 Jan 2020 23:31:50 GMT",4275.0
2,400.0,8.00,20.0,60.0,"Thu, 02 Jan 2020 13:57:17 GMT",True,"Thu, 02 Jan 2020 14:56:35 GMT",344.0
3,400.0,8.00,20.0,65.0,"Thu, 02 Jan 2020 14:00:03 GMT",True,"Thu, 02 Jan 2020 15:04:58 GMT",1117.0
4,400.0,16.00,40.0,504.0,"Thu, 02 Jan 2020 14:00:13 GMT",True,"Thu, 02 Jan 2020 22:24:01 GMT",334.0
...,...,...,...,...,...,...,...,...
56531,275.0,8.25,30.0,374.0,"Mon, 01 Jul 2019 19:13:10 GMT",True,"Tue, 02 Jul 2019 01:27:11 GMT",2643.0
56532,231.0,36.96,160.0,310.0,"Mon, 01 Jul 2019 19:43:58 GMT",True,"Tue, 02 Jul 2019 00:53:46 GMT",394.0
56533,300.0,12.00,40.0,381.0,"Mon, 01 Jul 2019 20:34:29 GMT",True,"Tue, 02 Jul 2019 02:55:30 GMT",2276.0
56534,400.0,32.00,80.0,36.0,"Mon, 01 Jul 2019 21:49:37 GMT",True,"Mon, 01 Jul 2019 22:25:12 GMT",1346.0


In [43]:
user_df.info()
user_df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56536 entries, 0 to 56535
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   WhPerMile           41309 non-null  float64
 1   kWhRequested        41309 non-null  float64
 2   milesRequested      41309 non-null  float64
 3   minutesAvailable    41309 non-null  float64
 4   modifiedAt          41309 non-null  object 
 5   paymentRequired     41309 non-null  object 
 6   requestedDeparture  41309 non-null  object 
 7   userID              41309 non-null  float64
dtypes: float64(5), object(3)
memory usage: 3.5+ MB


,WhPerMile,kWhRequested,milesRequested,minutesAvailable,userID
count,41309.000000,41309.000000,41309.000000,41309.000000,41309.000000
mean,363.481614,24.176610,71.077634,357.868237,1730.028662
std,102.904382,20.367215,59.569050,188.950229,2523.718804
min,50.000000,0.000000,0.000000,1.000000,1.000000
25%,290.000000,12.000000,30.000000,196.000000,438.000000
50%,360.000000,18.000000,50.000000,360.000000,766.000000
75%,400.000000,30.000000,100.000000,501.000000,1746.000000
max,2000.000000,215.320000,480.000000,1913.000000,19923.000000


Combine this with the rest of the data and drop the now redundant userInputs column

In [44]:
#Reset the index of both dataframes, because we have dropped some values
user_df.reset_index(drop=True, inplace=True)
charging_sessions_modified.reset_index(drop=True, inplace=True)
user_df.drop(columns=["userID"], inplace=True) #drop this so we dont duplicate it in the concatenated dataframe
charging_sessions_modified = pd.concat([charging_sessions_modified,user_df], axis = 1)
charging_sessions_modified.drop(columns= ["userInputs"], inplace= True )

In [45]:
charging_sessions_modified

,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,siteID,userID,combinedSpaceStationID,timeConnected,idleTime,chargingTime,weekday,month,time,WhPerMile,kWhRequested,milesRequested,minutesAvailable,modifiedAt,paymentRequired,requestedDeparture
0,2020-01-02 13:08:54+00:00,2020-01-02 19:11:15+00:00,2020-01-02 17:31:35+00:00,25.016,1,194.0,AG-3F30/1-1-179-810,21741.0,5980.0,15761.0,3,1,13:08:54,250.0,25.00,100.0,463.0,"Thu, 02 Jan 2020 13:09:39 GMT",True,"Thu, 02 Jan 2020 20:51:54 GMT"
1,2020-01-02 13:36:50+00:00,2020-01-02 22:38:21+00:00,2020-01-02 20:18:05+00:00,33.097,1,4275.0,AG-1F01/1-1-193-825,32491.0,8416.0,24075.0,3,1,13:36:50,280.0,70.00,250.0,595.0,"Thu, 02 Jan 2020 13:37:11 GMT",True,"Thu, 02 Jan 2020 23:31:50 GMT"
2,2020-01-02 13:56:35+00:00,2020-01-03 00:39:22+00:00,2020-01-02 16:35:06+00:00,6.521,1,344.0,AG-1F03/1-1-193-829,38567.0,29056.0,9511.0,3,1,13:56:35,400.0,8.00,20.0,60.0,"Thu, 02 Jan 2020 13:57:17 GMT",True,"Thu, 02 Jan 2020 14:56:35 GMT"
3,2020-01-02 13:59:58+00:00,2020-01-02 16:38:39+00:00,2020-01-02 15:18:45+00:00,2.355,1,1117.0,AG-1F04/1-1-193-820,9521.0,4794.0,4727.0,3,1,13:59:58,400.0,8.00,20.0,65.0,"Thu, 02 Jan 2020 14:00:03 GMT",True,"Thu, 02 Jan 2020 15:04:58 GMT"
4,2020-01-02 14:00:01+00:00,2020-01-02 22:08:40+00:00,2020-01-02 18:17:30+00:00,13.375,1,334.0,AG-1F06/1-1-193-819,29319.0,13870.0,15449.0,3,1,14:00:01,400.0,16.00,40.0,504.0,"Thu, 02 Jan 2020 14:00:13 GMT",True,"Thu, 02 Jan 2020 22:24:01 GMT"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56531,2019-07-01 19:13:11+00:00,2019-07-02 00:44:40+00:00,2019-07-01 21:13:56+00:00,6.048,1,2643.0,AG-3F23/1-1-179-797,19889.0,12644.0,7245.0,0,7,19:13:11,275.0,8.25,30.0,374.0,"Mon, 01 Jul 2019 19:13:10 GMT",True,"Tue, 02 Jul 2019 01:27:11 GMT"
56532,2019-07-01 19:43:46+00:00,2019-07-02 02:58:53+00:00,2019-07-02 02:43:53+00:00,41.047,1,394.0,AG-4F38/1-1-191-793,26107.0,900.0,25207.0,0,7,19:43:46,231.0,36.96,160.0,310.0,"Mon, 01 Jul 2019 19:43:58 GMT",True,"Tue, 02 Jul 2019 00:53:46 GMT"
56533,2019-07-01 20:34:30+00:00,2019-07-02 00:30:52+00:00,2019-07-01 21:27:44+00:00,5.390,1,2276.0,AG-3F15/1-1-179-813,14182.0,10988.0,3194.0,0,7,20:34:30,300.0,12.00,40.0,381.0,"Mon, 01 Jul 2019 20:34:29 GMT",True,"Tue, 02 Jul 2019 02:55:30 GMT"
56534,2019-07-01 21:49:12+00:00,2019-07-02 01:37:18+00:00,2019-07-01 23:14:33+00:00,8.399,1,1346.0,AG-3F18/1-1-179-798,13686.0,8565.0,5121.0,0,7,21:49:12,400.0,32.00,80.0,36.0,"Mon, 01 Jul 2019 21:49:37 GMT",True,"Mon, 01 Jul 2019 22:25:12 GMT"


In [46]:
#Split data into sites 1 and 2
site_1_data = charging_sessions_modified[charging_sessions_modified['siteID']==1].copy()
site_2_data = charging_sessions_modified[charging_sessions_modified['siteID']==2].copy()

In [47]:
site_1_data.info()
site_2_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28384 entries, 0 to 56535
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype              
---  ------                  --------------  -----              
 0   connectionTime          28384 non-null  datetime64[ns, UTC]
 1   disconnectTime          28384 non-null  datetime64[ns, UTC]
 2   doneChargingTime        28384 non-null  datetime64[ns, UTC]
 3   kWhDelivered            28384 non-null  float64            
 4   siteID                  28384 non-null  int64              
 5   userID                  27091 non-null  float64            
 6   combinedSpaceStationID  28384 non-null  object             
 7   timeConnected           28384 non-null  float64            
 8   idleTime                28384 non-null  float64            
 9   chargingTime            28384 non-null  float64            
 10  weekday                 28384 non-null  int32              
 11  month                   28384 non-null  int32 

In [48]:
site_1_data.reset_index(inplace=True)
site_2_data.reset_index(inplace=True)

In [49]:
# Drop the siteID, it's no longer needed
site_1_data.drop(columns=['siteID'], inplace=True)
site_2_data.drop(columns=['siteID'], inplace=True)

In [50]:
# Check for unique space/station id's so we know the maximum number of parking spaces available
site_1_data.nunique()

index                     28384
connectionTime            28275
disconnectTime            28338
doneChargingTime          28210
kWhDelivered              17173
userID                      499
combinedSpaceStationID       52
timeConnected             15864
idleTime                  15933
chargingTime              16894
weekday                       7
month                        12
time                      18054
WhPerMile                   144
kWhRequested                860
milesRequested               75
minutesAvailable            781
modifiedAt                26996
paymentRequired               1
requestedDeparture        27070
dtype: int64

In [51]:
site_2_data.nunique()

index                     28152
connectionTime            28113
disconnectTime            28116
doneChargingTime          28036
kWhDelivered              14399
userID                      455
combinedSpaceStationID       54
timeConnected             17936
idleTime                  13304
chargingTime              13690
weekday                       7
month                        12
time                      20896
WhPerMile                   110
kWhRequested                580
milesRequested               65
minutesAvailable            673
modifiedAt                14211
paymentRequired               1
requestedDeparture        14214
dtype: int64

In [52]:
site_1_data["connectionTime"].describe()

count                                  28384
mean     2019-11-25 14:02:36.180030976+00:00
min                2018-10-08 13:01:34+00:00
25%                2019-03-23 16:25:15+00:00
50%         2019-09-09 14:38:51.500000+00:00
75%      2020-05-03 16:18:13.249999872+00:00
max                2021-09-14 05:43:39+00:00
Name: connectionTime, dtype: object

In [53]:
site_1_data["disconnectTime"].describe()

count                                  28384
mean     2019-11-25 21:14:20.177388800+00:00
min                2018-10-09 00:05:33+00:00
25%         2019-03-23 21:59:39.500000+00:00
50%         2019-09-10 00:18:44.500000+00:00
75%                2020-05-03 20:52:27+00:00
max                2021-09-14 14:46:28+00:00
Name: disconnectTime, dtype: object

In [54]:
site_2_data["connectionTime"].describe()

count                                  28152
mean     2019-03-23 08:26:24.915210240+00:00
min                2018-04-25 13:45:10+00:00
25%         2018-08-20 10:28:28.500000+00:00
50%                2018-12-12 17:03:21+00:00
75%      2019-08-27 12:31:18.750000128+00:00
max                2021-09-13 23:11:12+00:00
Name: connectionTime, dtype: object

In [55]:
site_2_data["disconnectTime"].describe()

count                                  28152
mean     2019-03-23 14:18:31.080917760+00:00
min                2018-04-25 18:15:46+00:00
25%         2018-08-20 16:33:31.500000+00:00
50%         2018-12-13 01:32:46.500000+00:00
75%      2019-08-27 17:28:13.750000128+00:00
max                2021-09-14 02:22:25+00:00
Name: disconnectTime, dtype: object

In [56]:
# Create a dataframe with dates from the first to the last day of the charging sessions available. Because these differ for both sites, we need to do this for each respectively. Add a timedelta of 12 hours to site 2 disconnect, otherwise it would cut off the last day due to the hour of the timestamp
date_frame_site_1 = pd.DataFrame(pd.date_range(start=site_1_data["connectionTime"].min(), end = site_1_data["disconnectTime"].max()))
date_frame_site_2 = pd.DataFrame(pd.date_range(start=site_2_data["connectionTime"].min(), end = (site_2_data["disconnectTime"].max() + timedelta(hours = 12))))

In [57]:
date_frame_site_1

,0
0,2018-10-08 13:01:34+00:00
1,2018-10-09 13:01:34+00:00
2,2018-10-10 13:01:34+00:00
3,2018-10-11 13:01:34+00:00
4,2018-10-12 13:01:34+00:00
...,...
1068,2021-09-10 13:01:34+00:00
1069,2021-09-11 13:01:34+00:00
1070,2021-09-12 13:01:34+00:00
1071,2021-09-13 13:01:34+00:00


In [58]:
data = {'date': [], 'hour': []}
for row in date_frame_site_1.itertuples():
    for i in range(24):
        data["date"].append(row[1])
        data["hour"].append(i)
frame1 = pd.DataFrame(data)
frame1["weekday"] = frame1["date"].dt.day_of_week
frame1["month"] = frame1["date"].dt.month

data = {'date' : [], 'hour': []}
for row in date_frame_site_2.itertuples():
    for i in range(24):
        data["date"].append(row[1])
        data["hour"].append(i)
frame2 = pd.DataFrame(data)
frame2["weekday"] = frame2["date"].dt.day_of_week
frame2["month"] = frame2["date"].dt.month

In [59]:
frame2

,date,hour,weekday,month
0,2018-04-25 13:45:10+00:00,0,2,4
1,2018-04-25 13:45:10+00:00,1,2,4
2,2018-04-25 13:45:10+00:00,2,2,4
3,2018-04-25 13:45:10+00:00,3,2,4
4,2018-04-25 13:45:10+00:00,4,2,4
...,...,...,...,...
29731,2021-09-14 13:45:10+00:00,19,1,9
29732,2021-09-14 13:45:10+00:00,20,1,9
29733,2021-09-14 13:45:10+00:00,21,1,9
29734,2021-09-14 13:45:10+00:00,22,1,9


In [60]:
us_ca_holidays = holidays.country_holidays(country='US', subdiv='CA')

frame1["holiday"] = frame1["date"].apply(lambda x: x in us_ca_holidays)
frame2["holiday"] = frame2["date"].apply(lambda x: x in us_ca_holidays)

Since the data doesn't start at the beginning of the day and doesn't end at the end, we'll manually delete a few rows to fit it properly.
Both sites start in hour 13. Site 1 ends in hour 14 so we will have to drop the last 9 rows.
 Site 2 ends in hour 2, so we will drop the last 21.

In [61]:
frame1.drop(frame1.index[:13], inplace=True)
frame2.drop(frame2.index[:13], inplace=True)
frame2.drop(frame2.index[-9:], inplace=True)

In [63]:
weather = pd.read_csv("./data/original/weather_burbank_airport.csv")
weather

,city,timestamp,temperature,cloud_cover,cloud_cover_description,pressure,windspeed,precipitation,felt_temperature
0,Burbank,2018-01-01 08:53:00,9.0,33.0,Fair,991.75,9.0,0.0,8.0
1,Burbank,2018-01-01 09:53:00,9.0,33.0,Fair,992.08,0.0,0.0,9.0
2,Burbank,2018-01-01 10:53:00,9.0,21.0,Haze,992.08,0.0,0.0,9.0
3,Burbank,2018-01-01 11:53:00,9.0,29.0,Partly Cloudy,992.08,0.0,0.0,9.0
4,Burbank,2018-01-01 12:53:00,8.0,33.0,Fair,992.08,0.0,0.0,8.0
...,...,...,...,...,...,...,...,...,...
29239,Burbank,2021-01-01 03:53:00,13.0,33.0,Fair,986.81,0.0,0.0,13.0
29240,Burbank,2021-01-01 04:53:00,12.0,33.0,Fair,986.81,11.0,0.0,12.0
29241,Burbank,2021-01-01 05:53:00,12.0,33.0,Fair,987.47,9.0,0.0,12.0
29242,Burbank,2021-01-01 06:53:00,11.0,33.0,Fair,987.14,13.0,0.0,11.0


In [64]:
# We will only use temperature and precipitation from the weather dataset, so we'll drop everything else
weather.drop(columns=["city", 'cloud_cover', 'cloud_cover_description', 'pressure', 'windspeed', 'felt_temperature'], axis = 1, inplace=True)

In [65]:
weather

,timestamp,temperature,precipitation
0,2018-01-01 08:53:00,9.0,0.0
1,2018-01-01 09:53:00,9.0,0.0
2,2018-01-01 10:53:00,9.0,0.0
3,2018-01-01 11:53:00,9.0,0.0
4,2018-01-01 12:53:00,8.0,0.0
...,...,...,...
29239,2021-01-01 03:53:00,13.0,0.0
29240,2021-01-01 04:53:00,12.0,0.0
29241,2021-01-01 05:53:00,12.0,0.0
29242,2021-01-01 06:53:00,11.0,0.0


In [66]:
weather['timestamp'] = pd.to_datetime(weather['timestamp'])

In [67]:
# Add a new column dateHour, which represents the date with the current hour, we will merge into the site data on this later on. Drop the timestamp because we don't have any more use for it
weather["dateHour"] = weather["timestamp"].apply(lambda x: (str(x.year) + "-"
                                                            + str(x.month) +"-"
                                                            + str(x.day) +"+"
                                                            + str(x.hour)))
weather.drop(columns="timestamp")

,temperature,precipitation,dateHour
0,9.0,0.0,2018-1-1+8
1,9.0,0.0,2018-1-1+9
2,9.0,0.0,2018-1-1+10
3,9.0,0.0,2018-1-1+11
4,8.0,0.0,2018-1-1+12
...,...,...,...
29239,13.0,0.0,2021-1-1+3
29240,12.0,0.0,2021-1-1+4
29241,12.0,0.0,2021-1-1+5
29242,11.0,0.0,2021-1-1+6


In [68]:
# Add the dateHour column to both dataframes
frame1["dateHour"] = frame1["date"].apply(lambda x: (str(x.year) + "-"
                                                            + str(x.month) +"-"
                                                            + str(x.day) +"+"
                                                            ))
frame1["dateHour"] = frame1.apply(lambda x: '%s%s' % (x["dateHour"], x["hour"]), axis =1)

frame2["dateHour"] = frame2["date"].apply(lambda x: (str(x.year) + "-"
                                                            + str(x.month) +"-"
                                                            + str(x.day) +"+"
                                                            ))
frame2["dateHour"] = frame2.apply(lambda x: '%s%s' % (x["dateHour"], x["hour"]), axis =1)

In [69]:
# Function to return all hours between a start and end date in dateHour notation
def getHoursBetween(start, end):
    start = start.floor("h")
    end = end.floor("h")
    days= []
    for i in range(int((end-start).seconds/3600)+1):
            day = start+timedelta(hours=i)
            dateHour = str(day.year) + "-" + str(day.month) + "-" + str(day.day) + "+" + str(day.hour)
            days.append(dateHour)
    return days

In [70]:
# Iterate over all site 1 data, for each charging event, increment utilization by one for each hour. Keep track of the charging stations in stations, each station can have a max utilization of 1, meaning there is no difference if the station was occupied for 1 or 60 hour, or if one charging event ended and another started in the same hour
frame1["utilization"] = 0
frame1["stations"] = ""

for row in site_1_data.itertuples():
    connectionTime = row[2]
    disconnectTime = row[3]
    station = row[7]
    hours = getHoursBetween(connectionTime, disconnectTime)
    for hour in hours:
        hourRowIndex = frame1[frame1["dateHour"] == hour].index.tolist()[0]
        if not (str(frame1.at[hourRowIndex, "stations"]).__contains__(station)):
            frame1.at[hourRowIndex, "utilization"] += 1
            frame1.at[hourRowIndex, "stations"] += station +"_"
frame1['utilization'] /= 52
frame1.drop(columns="stations", inplace=True)

In [71]:
# Iterate over all site 2 data, for each charging event, increment utilization by one for each hour. Keep track of the charging stations in stations, each station can have a max utilization of 1, meaning there is no difference if the station was occupied for 1 or 60 hour, or if one charging event ended and another started in the same hour
frame2["utilization"] = 0
frame2["stations"] = ""

for row in site_2_data.itertuples():
    connectionTime = row[2]
    disconnectTime = row[3]
    station = row[7]
    hours = getHoursBetween(connectionTime, disconnectTime)
    for hour in hours:
        hourRowIndex = frame2[frame2["dateHour"] == hour].index.tolist()[0]
        if not (str(frame2.at[hourRowIndex, "stations"]).__contains__(station)):
            frame2.at[hourRowIndex, "utilization"] += 1
            frame2.at[hourRowIndex, "stations"] += station +"_"
frame2['utilization'] /= 54
frame2.drop(columns="stations", inplace = True)

In [82]:
frame1.to_pickle('./data/prepared/utilization_data_site1_complete.pkl')
frame2.to_pickle('./data/prepared/utilization_data_site2_complete.pkl')

In [73]:
frame1

,date,hour,weekday,month,holiday,dateHour,utilization
13,2018-10-08 13:01:34+00:00,13,0,10,True,2018-10-8+13,0.076923
14,2018-10-08 13:01:34+00:00,14,0,10,True,2018-10-8+14,0.365385
15,2018-10-08 13:01:34+00:00,15,0,10,True,2018-10-8+15,0.365385
16,2018-10-08 13:01:34+00:00,16,0,10,True,2018-10-8+16,0.365385
17,2018-10-08 13:01:34+00:00,17,0,10,True,2018-10-8+17,0.365385
...,...,...,...,...,...,...,...
25747,2021-09-14 13:01:34+00:00,19,1,9,False,2021-9-14+19,0.000000
25748,2021-09-14 13:01:34+00:00,20,1,9,False,2021-9-14+20,0.000000
25749,2021-09-14 13:01:34+00:00,21,1,9,False,2021-9-14+21,0.000000
25750,2021-09-14 13:01:34+00:00,22,1,9,False,2021-9-14+22,0.000000


For utilization prediction we will exclude any data after March 2020 due to the Corona pandemic. We argue, that due to lockdowns and a shift to homeoffice whereever possible, the rest of 2020 is not relevant to the situation we have today. While homeoffice is generally more accepted nowadays, we are beginning to see a lot of companies enforcing a return to the office.
While the data is beginning to normalize at the beginning of 2021, we do not have any weather data after this point. Attempts to find additional data for the burbank airport where unsuccessful. Due to both of these points, there will be a hard cutoff after March 2020.

In [74]:
# Drop data after onset of Corona

frame1 = frame1[frame1["date"]<"2020-04-01"]
frame2 = frame2[frame2["date"]<"2020-04-01"]

In [75]:
# drop duplicates in weather to ensure correct merge of both dataframes
weather.drop_duplicates(subset='dateHour', inplace=True)

In [76]:
# Merge weather data with the charging data
frame1 = frame1.merge(weather, left_on="dateHour", right_on="dateHour", how="left")
frame2 = frame2.merge(weather, left_on="dateHour", right_on="dateHour", how="left")

In [77]:
frame1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12971 entries, 0 to 12970
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype              
---  ------         --------------  -----              
 0   date           12971 non-null  datetime64[ns, UTC]
 1   hour           12971 non-null  int64              
 2   weekday        12971 non-null  int32              
 3   month          12971 non-null  int32              
 4   holiday        12971 non-null  bool               
 5   dateHour       12971 non-null  object             
 6   utilization    12971 non-null  float64            
 7   timestamp      12947 non-null  datetime64[ns]     
 8   temperature    12938 non-null  float64            
 9   precipitation  12947 non-null  float64            
dtypes: bool(1), datetime64[ns, UTC](1), datetime64[ns](1), float64(3), int32(2), int64(1), object(1)
memory usage: 823.5+ KB


In [78]:
frame2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16955 entries, 0 to 16954
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype              
---  ------         --------------  -----              
 0   date           16955 non-null  datetime64[ns, UTC]
 1   hour           16955 non-null  int64              
 2   weekday        16955 non-null  int32              
 3   month          16955 non-null  int32              
 4   holiday        16955 non-null  bool               
 5   dateHour       16955 non-null  object             
 6   utilization    16955 non-null  float64            
 7   timestamp      16915 non-null  datetime64[ns]     
 8   temperature    16905 non-null  float64            
 9   precipitation  16915 non-null  float64            
dtypes: bool(1), datetime64[ns, UTC](1), datetime64[ns](1), float64(3), int32(2), int64(1), object(1)
memory usage: 1.1+ MB


There are a few datapoints where we dont have the weather data available. Because we are working with weather data, we don't expect things to change abruptly. Temperature will likely drop from 14->12->10 instead of 14->10->10. We will use pandas interpolate to model this change for missing values.

In [79]:
frame1.interpolate(inplace=True)
frame2.interpolate(inplace=True)

C:\Users\Ann\AppData\Local\Temp\ipykernel_24272\3557361287.py:1: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  frame1.interpolate(inplace=True)
C:\Users\Ann\AppData\Local\Temp\ipykernel_24272\3557361287.py:2: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  frame2.interpolate(inplace=True)


In [80]:
frame1.drop(columns=['date','dateHour','timestamp'], inplace=True)
frame2.drop(columns=['date','dateHour','timestamp'], inplace=True)

In [81]:
frame1.to_pickle('./data/prepared/utilization_data_site1.pkl')
frame2.to_pickle('./data/prepared/utilization_data_site2.pkl')